In [5]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')  # Utiliser un modèle de base, ou remplacer par un modèle personnalisé


In [2]:
#### ANALYSE DES PIXELS FRAME PAR FRAME
import cv2
import numpy as np

video_path = 'penalty5.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Erreur: Impossible de lire la vidéo.")
    exit()

width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

print(f"Dimensions de la vidéo : {width}x{height}, FPS : {fps}")

output_path = 'penalty5_pixel.mp4'  
fourcc = cv2.VideoWriter_fourcc(*'H264')  # Utiliser XVID comme codec
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

frame_count = 0
prev_frame = None

while True:
    # Lire la frame actuelle
    ret, frame = cap.read()

    # Si la vidéo est terminée, quitter
    if not ret:
        print("Fin de la vidéo ou erreur de lecture.")
        break

    # Convertir en niveaux de gris pour simplifier la détection de variations
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    if prev_frame is not None:
        # Calculer la différence absolue entre la frame actuelle et la précédente
        diff = cv2.absdiff(prev_frame, gray_frame)

        # Appliquer un seuil pour créer une image binaire (pour détecter des changements significatifs)
        _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)

        # Trouver les contours des zones où il y a des différences
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Dessiner les contours sur la frame originale (en couleur)
        annotated_frame = frame.copy()
        cv2.drawContours(annotated_frame, contours, -1, (0, 255, 0), 2)

        # Si une variation importante est détectée, ajouter une annotation texte
        if np.sum(diff) > 5000:  # seuil arbitraire
            cv2.putText(annotated_frame, f"Variation detected at frame {frame_count}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)

        # Enregistrer la frame annotée dans le fichier vidéo
        out.write(annotated_frame)

    # Mettre à jour la frame précédente
    prev_frame = gray_frame
    frame_count += 1

# Libérer les ressources
cap.release()
out.release()
cv2.destroyAllWindows()

print("Video processed and saved successfully.")


Dimensions de la vidéo : 1920x1080, FPS : 30
Fin de la vidéo ou erreur de lecture.
Video processed and saved successfully.
